# Bacteria Parser
This parses the bacteria fasta files into a panda dataframe

## Set up

In [1]:
import os
import re
import csv
import glob
import sqlite3
import pandas as pd
from Bio import SeqIO

#### Point this to the folder with the fasta files

In [3]:
fasta_dir = "ncbi_dataset/ncbi_dataset/data"

In [4]:
# Looking for folders starting with G (wildcard) and grabbing the path to its protein.faa file
fasta_paths = glob.glob(os.path.join(fasta_dir, "G*/protein.faa"))

In [4]:
# Checking for accurate retrieval
print(fasta_paths[:20])

['ncbi_dataset/ncbi_dataset/data/GCF_900112345.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_029869925.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_013868055.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_001047375.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_001528745.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_000007605.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_045833485.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_035285365.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_042685625.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_000723885.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_024347455.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_000708045.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_014495845.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_900167905.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_003716875.1/protein.faa', 'ncbi_dataset/ncbi_dataset/data/GCF_001547735.1/protein.faa', 'ncbi_d

## Creating pandas dataframe from fastas

### Creating sql table

In [5]:
conn = sqlite3.connect("bacteria.db")

# Create table scheme with sql
conn.execute("""
CREATE TABLE IF NOT EXISTS sequences (
  genome_id TEXT,
  protein_id TEXT,
  protein_seq TEXT,
  taxonomic_label TEXT,
  description TEXT
)
""")

### Extracting ids and sequences from MULTIPLE fasta

In [6]:
def parse_fasta (path):

    # Scan through every record in fasta
    rows = []
    for record in SeqIO.parse(path, "fasta"):

        # name of the folder containing fasta
        folder_name = os.path.basename(os.path.dirname(path))
        
        full_header = record.description

        # extract organism name inside brackets using regex
        match = re.search(r'\[(.+?)\]', full_header)
        organism = match.group(1) if match else ""
        
        # create substring without bracketed part
        no_id = full_header[len(record.id):].lstrip(" :")        
        pre_description = re.sub(r'\s*\[.+?\]', "", no_id)


        rows.append({
            "genome_id": folder_name,
            "protein_id": record.id,
            "protein_seq": str(record.seq),
            "taxonomic_label": organism,    
            "description": pre_description
        })

    # print (*rows);
    return rows;
    

### Parse all fasta files and append to a Panda dataframe

In [7]:

for path in fasta_paths:
   rows = parse_fasta(path)
    
   pd.DataFrame(rows).to_sql("sequences", conn, if_exists="append", index=False)


conn.commit()
conn.close()
    

### Extracting protein id set from SINGLE fasta

In [3]:
def parse_fasta (path):

    # Scan through every record in fasta
    # Empty list to hold sequences
    seq_list = []

    for record in SeqIO.parse(path, "fasta"):

        sequence = str(record.seq)
        
        seq_list.append(sequence)
   
    return seq_list;


# Search through fasta and store list into file

seq_list = parse_fasta('ncbi_dataset/ncbi_dataset/90/clustered_proteins_rep_seq.fasta')


with open('seq_list_90.txt', 'w') as file:
    # For better readibility list each protein id on new line
    for item in seq_list:
        file.write(item + '\n')


### Checking Database

In [2]:
# connect to database
conn = sqlite3.connect("bacteria.db")

# query for first 10000 in sequences table
chunk = pd.read_sql_query("SELECT * FROM sequences LIMIT 1000000", conn)

# Display the query
display(chunk)


conn.close()

,genome_id,protein_id,protein_seq,taxonomic_label,description
0,GCF_900112345.1,WP_004008316.1,MKVNPSVKPICDKCKVIRRHGRVMVICENPRHKQRQG,Bacteria,MULTISPECIES: 50S ribosomal protein L36
1,GCF_900112345.1,WP_008360994.1,MAGQKIRIRLKAYDHEVIDTSARKIVDTVTRTGAKVAGPVPLPTEK...,Actinomycetes,MULTISPECIES: 30S ribosomal protein S10
2,GCF_900112345.1,WP_008361020.1,MAKTALKVKAARKPKYAVRAYTRCQRCGRPKAVFRKFGLCRICLRE...,Nocardioides,MULTISPECIES: type Z 30S ribosomal protein S14
3,GCF_900112345.1,WP_011757324.1,MIQQESRLKVADNTGAKEILCIRVLGGSGRRYAGIGDVIVATVKDA...,Nocardioidaceae,MULTISPECIES: 50S ribosomal protein L14
4,GCF_900112345.1,WP_017933175.1,MGSVIKKRRKRMAKKKHRKLLKKTRVARRKLGK,Nocardioides,MULTISPECIES: 30S ribosomal protein bS22
...,...,...,...,...,...
999995,GCF_021391395.1,WP_233783998.1,MAEPPAPAAPQASRTCFWGSQVTGFSDAGPDRAILNIGQRESWELT...,Sphingomonas cannabina,DUF6491 family protein
999996,GCF_021391395.1,WP_233783999.1,MNEDHDRPIPQSPLDAAPAATRRLLAETPLGTVEIPRPPLETLPYR...,Sphingomonas cannabina,hypothetical protein
999997,GCF_021391395.1,WP_233784000.1,MRFRRLGRTGLNLSELTIGTAGLARVAPEQARAAITLALDRGVNAV...,Sphingomonas cannabina,aldo/keto reductase
999998,GCF_021391395.1,WP_233784001.1,MEIANHLFYGDNLDVLREHIPDESVDLIYLDPPFNSNAGYNVLFKA...,Sphingomonas cannabina,DNA methyltransferase


In [6]:
# connect to database
conn = sqlite3.connect("bacteria.db")

cur = conn.cursor()

# Count number of 
cur.execute("SELECT COUNT(*) FROM sequences")
count = cur.fetchone()[0]

print(f"Total rows in sequences: {count}")

conn.close()

Total rows in sequences: 86523087


### Export into csv

In [ ]:
conn = sqlite3.connect("bacteria.db")
cur  = conn.cursor()

# run the query once
cur.execute("SELECT * FROM sequences;")

# grab the header
header = [col[0] for col in cur.description]

with open("bacteriaDB.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(header)
    
    # fetch & write in batches of 10 000
    while True:
        batch = cur.fetchmany(10_000)
        if not batch:
            break
        writer.writerows(batch)

conn.close()
print("Done")

In [ ]:






SELECT COUNT(*) FROM